In [40]:
import os

In [ ]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
%pwd

'/Users/sohamkhopkar/Desktop/LangChain Projects/Final-major/Github'

In [3]:
!mkdir test_repo

In [4]:
repo_path = "test_repo/"

In [5]:
Repo.clone_from("https://github.com/Sohamm31/Final-major", to_path=repo_path)


<git.repo.base.Repo '/Users/sohamkhopkar/Desktop/LangChain Projects/Final-major/Github/test_repo/.git'>

In [11]:

loader = GenericLoader.from_filesystem("/Users/sohamkhopkar/Desktop/LangChain Projects/Final-major/Github/test_repo",
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [12]:
repo_path = "test_repo/"
documents = loader.load()

In [13]:
documents

[Document(metadata={'source': '/Users/sohamkhopkar/Desktop/LangChain Projects/Final-major/Github/test_repo/mock_interview.py', 'language': <Language.PYTHON: 'python'>}, page_content='from langchain_core.prompts import PromptTemplate\nfrom langchain.chat_models import ChatOpenAI\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_community.document_loaders import PyPDFLoader\nfrom langchain.output_parsers import StructuredOutputParser,ResponseSchema\nimport speech_recognition as sr\nimport threading\nimport pyttsx3\nimport time\nimport os\nfrom dotenv import load_dotenv\nload_dotenv() \n\napi_key = os.getenv(\'OPENROUTESERVICE_API_KEY\')\nrecognizer = sr.Recognizer()\ntts = pyttsx3.init()\n\ntts.setProperty(\'rate\', 180)  \nvoices = tts.getProperty(\'voices\')\ntts.setProperty(\'volume\', 1.0)\ntts.setProperty(\'voice\', voices[117].id)  \n\n\nloader = PyPDFLoader("Final-major/Soham Resume TnP Updated May.pdf")\npdf = loader.load()\nall_content = [doc.page_conten

In [14]:
len(documents)

1

In [15]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [16]:
texts = documents_splitter.split_documents(documents)

In [17]:

len(texts)

5

In [39]:
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
api_key = os.getenv('OPENROUTESERVICE_API_KEY')

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [20]:
embeddings = download_hugging_face_embeddings()


/var/folders/vr/k31rtn_168v7kvg7k8xq_v540000gn/T/ipykernel_54075/4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21

In [23]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')

In [24]:
vectordb.persist()


/var/folders/vr/k31rtn_168v7kvg7k8xq_v540000gn/T/ipykernel_54075/1246081259.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [42]:
llm = ChatOpenAI(
    model_name="deepseek/deepseek-r1-0528-qwen3-8b:free",  # You can use any OpenRouter-supported model
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1",  # Important for OpenRouter!
)


In [33]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [34]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

In [37]:
question = "Summarise the entire project with its key functionalities"

In [38]:
result = qa(question)
print(result['answer'])

I can see you're interested in understanding the entire structure and functionality of the project, which appears to be an interactive interview simulation system.

This seems to be an interview practice system that:
1. Takes a resume (loaded from PDF)
2. Generates interview questions based on the resume content
3. Conducts a voice-based interview using speech recognition and text-to-speech
4. Collects and organizes interview responses
5. Generates structured feedback categorized by different assessment areas

The key functionalities include:
- Resume parsing and question generation
- Voice-based interview process using speech recognition and synthesis
- Structured response collection
- Automated interview summary generation
- Categorized feedback generation
- Structured output parsing

This type of system could be useful for interview preparation, job candidate assessment, or educational purposes where verbal responses need to be captured and evaluated.
